In [2]:
import sys
from os import remove, rename
from os.path import exists
from viresclient._wps.wps_vires import ViresWPS10Service
from viresclient._wps.environment import JINJA2_ENVIRONMENT
from viresclient._wps.time_util import parse_datetime, parse_duration, encode_duration, timedelta, datetime
from viresclient._client import ProgressBarProcessing, ProgressBarDownloading, ClientRequest

wps_proxy = ViresWPS10Service(
    url="https://staging.viresdisc.vires.services/openows"
)

request_template = JINJA2_ENVIRONMENT.get_template(
    "vires_fetch_filtered_data_async.xml"
)

spcectraft_to_collections_mapping = {
    "A": "SW_OPER_MAGA_LR_1B",
    "B": "SW_OPER_MAGB_LR_1B",
    "C": "SW_OPER_MAGC_LR_1B",
}

def chop_timerange_by_months(start_time, end_time):
    year, month = start_time.year, start_time.month
    window_end = start_time
    while window_end < end_time:
        next_year, next_month = year, month + 1
        if next_month > 12:
            next_year += 1
            next_month -= 12
        window_start = max(start_time, datetime(year, month, 1))
        window_end = min(end_time, datetime(next_year, next_month, 1) - timedelta(microseconds=1))
        yield (window_start, window_end)
        year, month = next_year, next_month


In [ ]:
###################################################################################
# set the parameters and press Ctrl-ENTER

collection = "SW_OPER_MAGA_LR_1B" # or "SW_OPER_MAGB_LR_1B" or "SW_OPER_MAGC_LR_1B"
start = "2016-01-01" # included
end = "2017-01-01" # not included
#sampling = "PT60S" # set to None to use the original sampling
sampling = None
spacecraft = 'A'
models = ["CHAOS-6-Combined"]
variables=[
    "B_NEC_res_CHAOS-6-Combined", "B_NEC", "B_NEC_CHAOS-6-Combined",
    "F_res_CHAOS-6-Combined", "F", "F_CHAOS-6-Combined",
    'dF_AOCS', 'dF_other', 'F_error', 'B_VFM', 'dB_Sun',
    'dB_AOCS', 'dB_other', 'B_error', 'q_NEC_CRF', 'Att_error', 'Flags_F',
    'Flags_B', 'Flags_q', 'Flags_Platform', 'ASM_Freq_Dev',
    'Kp', 'Dst', 'F10_INDEX', 'OrbitNumber', 'QDLat', 'QDLon', 'QDBasis', 'MLT',
    'SunDeclination', 'SunHourAngle', 'SunRightAscension',
    'SunAzimuthAngle', 'SunZenithAngle', 'SunLongitude',
]

output_filename_template = "CHAOS6_residuals_{spacecraft}_{start}_{end}.cdf"

###################################################################################

start_time = parse_datetime(start)
end_time = max(start_time, parse_datetime(end) + timedelta(microseconds=-1))

# split time into monthly chunks
# note that VirES will not allow more than 50 days of 1s samped data

time_windows = list(chop_timerange_by_months(start_time, end_time))
output_filenames = [
    output_filename_template.format(
        spacecraft=spacecraft,
        start=chunk_start_time.strftime("%Y%m%d-%H%M%S"),
        end=chunk_end_time.strftime("%Y%m%d-%H%M%S"),
    ) for chunk_start_time, chunk_end_time in time_windows
]

# validate sampling step
if sampling:
    parse_duration(sampling)
    
# prepare collection identifiers
collections = {
    spacecraft: [spcectraft_to_collections_mapping[spacecraft]]
}

# print request summary
print("request summary:")
print("\toutput_filenames:")
for filename in output_filenames:
    print("\t\t%s" % filename)
print("\tstart time: %sZ" % start_time.isoformat())
print("\tend time:   %sZ" % end_time.isoformat())
print("\tsampling:   %s" % (sampling or "original data sampling"))
print("\tcollections:")
for spacecraft, colls in collections.items():
    print("\t%s: %s" % (spacecraft, ", ".join(colls)))
print("\tmodels:")
for model in models:
    print("\t\t%s" % model)
print("\tvariables:")
for variable in variables:
    print("\t\t%s" % variable)
print()

# download data by chunks
for (chunk_start_time, chunk_end_time), output_filename in zip(time_windows, output_filenames):
    print("processing %s" % output_filename)
    sys.stdout.flush()

    # XML WPS request
    request = request_template.render(
        begin_time=chunk_start_time,
        end_time=chunk_end_time,
        sampling_step=sampling,
        variables=variables,
        collection_ids=collections,
        model_ids=models,
        response_type="application/x-cdf",
    )
    
    # download the data
    if not exists(output_filename):
        tmp_filename = output_filename + ".tmp"
        try:
            with open(tmp_filename, "wb") as fout:
                response_handler=ClientRequest._response_handler(fout)
                with ProgressBarProcessing() as progress_bar:
                    wps_proxy.retrieve_async(
                        request.encode('UTF-8'),
                        handler=response_handler,
                        status_handler=progress_bar.update
                    )
        except Exception as error:
            if exists(tmp_filename):
                remove(tmp_filename)
            print("ERROR: %s faled: %s" % (output_filename, error), file=sys.stderr)
        else:
            if exists(tmp_filename):
                rename(tmp_filename, output_filename)
    else:
        print("%s exists, skipped ... " % output_filename)

request summary:
	output_filenames:
		CHAOS6_residuals_A_20160101-000000_20160131-235959.cdf
		CHAOS6_residuals_A_20160201-000000_20160229-235959.cdf
		CHAOS6_residuals_A_20160301-000000_20160331-235959.cdf
		CHAOS6_residuals_A_20160401-000000_20160430-235959.cdf
		CHAOS6_residuals_A_20160501-000000_20160531-235959.cdf
		CHAOS6_residuals_A_20160601-000000_20160630-235959.cdf
		CHAOS6_residuals_A_20160701-000000_20160731-235959.cdf
		CHAOS6_residuals_A_20160801-000000_20160831-235959.cdf
		CHAOS6_residuals_A_20160901-000000_20160930-235959.cdf
		CHAOS6_residuals_A_20161001-000000_20161031-235959.cdf
		CHAOS6_residuals_A_20161101-000000_20161130-235959.cdf
		CHAOS6_residuals_A_20161201-000000_20161231-235959.cdf
	start time: 2016-01-01T00:00:00Z
	end time:   2016-12-31T23:59:59.999999Z
	sampling:   original data sampling
	collections:
	A: SW_OPER_MAGA_LR_1B
	models:
		CHAOS-6-Combined
	variables:
		B_NEC_res_CHAOS-6-Combined
		B_NEC
		B_NEC_CHAOS-6-Combined
		F_res_CHAOS-6-Combined
		F
	

Processing:   75%|███████▌  |  [ Elapsed: 06:58, Remaining: 02:11 ]